In [ ]:
#code to identify those in the eCare Home cohort who have ever been identified with a diabetes mellitus code for efalls
create  or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efalls_diabetes_mellitus` as (
#bring all snomed and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  snomedcode,
  ctv3code,
  dateevent
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    cast(j.first_episodestartdate as datetime) first_episodestartdate,
    j.gender_source_value as gender
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#identify all individuals with code for Diabetes mellitus within 5 years for those codes with a time constraint of 5 years prior to caer home admission
dm_time_dep_snomed as (
  select
  person_id, 
  snomedcode, 
  ctv3code,
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Diabetes mellitus' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND  datetime_diff(first_episodestartdate,dateevent,year) <=5),
#identify all individuals with code for Diabetes mellitus within 5 years for those codes with a time constraint of 5 years prior to care home admission
dm_time_dep_ctv as (
  select
  person_id, 
  snomedcode,
  ctv3code, 
  dateevent,
  from a_master
  where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Diabetes mellitus' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate AND  datetime_diff(first_episodestartdate,dateevent,year) <=5),
#identify all individuals with Diabetes mellitus snomed codes  prior to caer home admission without time constraint
dm_time_nondep_snomed as (
  select
  person_id, 
  snomedcode, 
  ctv3code,
  dateevent,
  from a_master
  where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Diabetes mellitus' and TimeConstraintyears is null) AND dateevent < first_episodestartdate
  ),

#identify all individuals with Diabetes mellitus snomed codes  prior to caer home admission without time constraint
dm_time_nondep_ctv as (
  select
  person_id, 
  snomedcode,
  ctv3code, 
  dateevent,
  from a_master
  where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Diabetes mellitus' and TimeConstraintyears is null) AND dateevent < first_episodestartdate
  ),

dm_union as (
  select
  * from dm_time_dep_ctv

  union all

  select
  * from dm_time_dep_snomed

  union all 

  select
  * from dm_time_nondep_ctv

  union all 

  select
  * from dm_time_nondep_snomed
),

#sequences them to find the initial code within 5 years 
dm_union_seq as (
select
*,
row_number() over (partition by person_id order by dateevent desc) as code_seq
from dm_union)
#identifies all individuals in the CB cohort who have ever fulfilled the Diabetes mellitus code

select
person_id, snomedcode, ctv3code,dateevent as diagnosis_date,
case when person_id is not null then 'yes' end as diabetes_mellitus
from dm_union_seq
where code_seq = 1 
order by person_id)


